In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("C:/Users/Emre/Desktop/customer_behavior_train_useridTR.csv")

In [3]:
df.shape

(455401, 20)

In [14]:
df["satin_aldi"].value_counts(normalize=True)

satin_aldi
0    0.958074
1    0.041926
Name: proportion, dtype: float64

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 455401 entries, 0 to 455400
Data columns (total 20 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   userid                   455401 non-null  object
 1   sepet_ikonuna_tiklama    455401 non-null  int64 
 2   listeden_sepet_ekleme    455401 non-null  int64 
 3   detaydan_sepet_ekleme    455401 non-null  int64 
 4   hesap_sayfasina_tiklama  455401 non-null  int64 
 5   kampanyaya_tiklama       455401 non-null  int64 
 6   favoriye_ekleme          455401 non-null  int64 
 7   beden_secimi             455401 non-null  int64 
 8   mini_sepet_kapatma       455401 non-null  int64 
 9   teslimat_detay_bakma     455401 non-null  int64 
 10  iade_detay_bakma         455401 non-null  int64 
 11  giris_yapma              455401 non-null  int64 
 12  odeme_sayfasi_gorme      455401 non-null  int64 
 13  beden_tablosu_gorme      455401 non-null  int64 
 14  teslimat_bilgisi_gor

In [5]:
df.groupby(["satin_aldi", "beden_tablosu_gorme"]).size()

satin_aldi  beden_tablosu_gorme
0           0                      436152
            1                         156
1           0                       19072
            1                          21
dtype: int64

In [6]:
df = df.drop(columns="beden_tablosu_gorme") # Nadir olduğu için modelden çıkartıldı

In [7]:
kimlik_sutunu = ["userid"]

In [8]:
target = ["satin_aldi"]

In [9]:
df_corr = ["sepet_ikonuna_tiklama", "listeden_sepet_ekleme", "detaydan_sepet_ekleme", "hesap_sayfasina_tiklama", "kampanyaya_tiklama", "favoriye_ekleme", "beden_secimi", "mini_sepet_kapatma", "teslimat_detay_bakma", "iade_detay_bakma", "giris_yapma", "odeme_sayfasi_gorme", "teslimat_bilgisi_gorme", "cihaz_mobil", "cihaz_pc", "cihaz_tablet", "geri_donen_kullanici", "satin_aldi"]

In [10]:
df[df_corr].corr()["satin_aldi"]

sepet_ikonuna_tiklama      0.428334
listeden_sepet_ekleme      0.287666
detaydan_sepet_ekleme      0.414420
hesap_sayfasina_tiklama    0.057279
kampanyaya_tiklama         0.056533
favoriye_ekleme            0.023516
beden_secimi               0.154867
mini_sepet_kapatma         0.140011
teslimat_detay_bakma       0.798720
iade_detay_bakma           0.059484
giris_yapma                0.665556
odeme_sayfasi_gorme        0.708986
teslimat_bilgisi_gorme     0.031461
cihaz_mobil               -0.042907
cihaz_pc                   0.049208
cihaz_tablet               0.016939
geri_donen_kullanici       0.060295
satin_aldi                 1.000000
Name: satin_aldi, dtype: float64

In [11]:
feature_sutunlari = df.drop(["satin_aldi","userid"],axis=1)

In [12]:
feature_sutunlari.head(5)

,sepet_ikonuna_tiklama,listeden_sepet_ekleme,detaydan_sepet_ekleme,hesap_sayfasina_tiklama,kampanyaya_tiklama,favoriye_ekleme,beden_secimi,mini_sepet_kapatma,teslimat_detay_bakma,iade_detay_bakma,giris_yapma,odeme_sayfasi_gorme,teslimat_bilgisi_gorme,cihaz_mobil,cihaz_pc,cihaz_tablet,geri_donen_kullanici
0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
4,0,1,0,0,0,0,1,0,1,0,1,1,0,0,0,1,0


In [ ]:
# Eğitim / test ayrımı ve Logistic Regression, LightGBM, XGBoost modellerinin eğitimi

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from sklearn.linear_model import LogisticRegression

# Gerekirse aşağıdaki satırın yorumunu kaldırarak paketleri yükleyebilirsiniz
# !pip install lightgbm xgboost

from lightgbm import LGBMClassifier
import xgboost as xgb

# Özellik ve hedef ayrımı
X = df.drop(columns=["satin_aldi", "userid"])
y = df["satin_aldi"]

# Eğitim / test ayrımı
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y,
)

print("Egitim seti boyutu:", X_train.shape)
print("Test seti boyutu:", X_test.shape)

# 1) Logistic Regression
logreg = LogisticRegression(max_iter=1000, class_weight="balanced")
logreg.fit(X_train, y_train)

y_pred_lr = logreg.predict(X_test)
y_proba_lr = logreg.predict_proba(X_test)[:, 1]

print("\n=== Logistic Regression ===")
print("Accuracy:", accuracy_score(y_test, y_pred_lr))
print("ROC-AUC:", roc_auc_score(y_test, y_proba_lr))
print(classification_report(y_test, y_pred_lr))


# 2) LightGBM (sklearn wrapper - LGBMClassifier)
lgbm_clf = LGBMClassifier(
    n_estimators=500,
    learning_rate=0.05,
    num_leaves=31,
    subsample=0.8,
    colsample_bytree=0.9,
    objective="binary",
    subsample_freq=5,
    n_jobs=-1,
)

lgbm_clf.fit(
    X_train,
    y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    eval_metric="auc",
    verbose=False,
)

y_proba_lgb = lgbm_clf.predict_proba(X_test)[:, 1]
y_pred_lgb = (y_proba_lgb > 0.5).astype(int)

print("\n=== LightGBM (LGBMClassifier) ===")
print("Accuracy:", accuracy_score(y_test, y_pred_lgb))
print("ROC-AUC:", roc_auc_score(y_test, y_proba_lgb))
print(classification_report(y_test, y_pred_lgb))


# 3) XGBoost
xgb_model = xgb.XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="binary:logistic",
    eval_metric="auc",
    tree_method="hist",
    n_jobs=-1,
)

xgb_model.fit(
    X_train,
    y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    verbose=False,
)

y_proba_xgb = xgb_model.predict_proba(X_test)[:, 1]
y_pred_xgb = (y_proba_xgb > 0.5).astype(int)

print("\n=== XGBoost ===")
print("Accuracy:", accuracy_score(y_test, y_pred_xgb))
print("ROC-AUC:", roc_auc_score(y_test, y_proba_xgb))
print(classification_report(y_test, y_pred_xgb))

